In [1]:
import numpy as np
import librosa
import os
from glob import glob 
import shutil
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd
import json

In [2]:
os.chdir('C:\\Users\Alec\Desktop\ESC50\\audio')

In [3]:
files = glob('*.wav')

In [4]:
destination = "C:\\Users\Alec\Desktop\SIT_acoustic\\anechoic"

In [6]:
print(len(files))
count=0
for file in files:
    shutil.copy(file, destination)
    count+=1
print(f"Copied {count} files to {destination}")

544
Copied 544 files to C:\Users\Alec\Desktop\SIT_acoustic\anechoic


In [7]:
%%time
os.chdir("C:\\Users\Alec\Desktop\SIT_acoustic\\anechoic")

filesToProcess = glob('*.wav')

for w in filesToProcess:
    # Load .wav file using librosa
    y, sr = librosa.load(w, sr=44100)
    
    # Calculate number of 1-second chunks
    duration = librosa.get_duration(y=y, sr=sr)
    num_chunks = int(np.ceil(duration))
    
    # Iterate through each chunk
    for i in range(num_chunks):
        # Set the start and end time for the chunk
        start_time = i
        end_time = min(i + 1, duration)
        
        # Extract audio segment for the chunk
        y_chunk = y[int(start_time * sr):int(end_time * sr)]
        
        # Create output file name
        output_file = f"{os.path.splitext(w)[0]}_chunk{i+1}.wav"
        
        # Write chunk data to a new .wav file using soundfile
        sf.write(output_file, y_chunk, sr)
        
    
    # Delete original .wav file
    os.remove(w)
    print(f"Finished processing {w}, the original file has been deleted")

Finished processing DJIM600-FT-A-2426_20191126.wav, the original file has been deleted
Finished processing DJIM600-FT-A-2520_20191126.wav, the original file has been deleted
Finished processing DJIM600-FT-B-2426_20191126.wav, the original file has been deleted
Finished processing DJIM600-FT-B-2520_20191126.wav, the original file has been deleted
Finished processing DJIM600-FT-C-2426_20191126.wav, the original file has been deleted
Finished processing DJIM600-FT-C-2520_20191126.wav, the original file has been deleted
Finished processing DJIM600-FT-D-2426_20191126.wav, the original file has been deleted
Finished processing DJIM600-FT-D-2520_20191126.wav, the original file has been deleted
Finished processing DJIM600-HT-A-2216_20191126.wav, the original file has been deleted
Finished processing DJIM600-HT-A-2407_20191126.wav, the original file has been deleted
Finished processing DJIM600-HT-A-2499_20191126.wav, the original file has been deleted
Finished processing DJIM600-HT-B-2216_20191

In [8]:
from librosa.util import fix_length
chunks = glob('*.wav')
sr = 44100 
targetDuration = 1
for c in chunks:
    y, sr = librosa.load(c, sr=44100)
    duration = librosa.get_duration(y=y, sr=sr)
    if duration <= 0.05:
        os.remove(c)
        print(f'Removed file {c} which had duration {duration}')
    elif duration < 1:
        padded = fix_length(y, size=targetDuration*sr)
        sf.write(c, padded, sr)
        print(f'Padded {c}, new duration = {librosa.get_duration(y=padded, sr=44100)}')

Padded DJIM600-FT-A-2426_20191126_chunk23.wav, new duration = 1.0
Padded DJIM600-FT-A-2520_20191126_chunk21.wav, new duration = 1.0
Padded DJIM600-FT-B-2426_20191126_chunk23.wav, new duration = 1.0
Padded DJIM600-FT-B-2520_20191126_chunk21.wav, new duration = 1.0
Padded DJIM600-FT-C-2426_20191126_chunk23.wav, new duration = 1.0
Padded DJIM600-FT-C-2520_20191126_chunk21.wav, new duration = 1.0
Padded DJIM600-FT-D-2426_20191126_chunk23.wav, new duration = 1.0
Padded DJIM600-FT-D-2520_20191126_chunk21.wav, new duration = 1.0
Padded DJIM600-HT-A-2216_20191126_chunk17.wav, new duration = 1.0
Padded DJIM600-HT-A-2407_20191126_chunk18.wav, new duration = 1.0
Padded DJIM600-HT-A-2499_20191126_chunk20.wav, new duration = 1.0
Padded DJIM600-HT-B-2216_20191126_chunk17.wav, new duration = 1.0
Padded DJIM600-HT-B-2407_20191126_chunk18.wav, new duration = 1.0
Padded DJIM600-HT-B-2499_20191126_chunk20.wav, new duration = 1.0
Padded DJIM600-HT-C-2216_20191126_chunk17.wav, new duration = 1.0
Padded DJI

In [4]:
chunks = glob('*.wav')
for c in chunks:
    y, sr = librosa.load(c, sr=44100)
    duration = librosa.get_duration(y=y, sr=sr)
    if duration != 1:
        print(f'{c} has duration {duration} and not duration 1')

In [19]:
# Function to determine prefix number based on file name
def get_prefix_number(filename):
    # Define your prefixes and their corresponding numbers
    prefixes = {
        'DJIM600': 0,
        'DJIS1000': 1,
        'DJIS1000LOUD': 2,
        'MAVIC2PRO': 3,
        'MAVICPRO': 4,
        'MAVICPROPLAT': 5,
        'PHANTOM4': 6,
        'PHANTOM4QP': 7,
    }
    # Iterate through prefixes to find a match
    for prefix, number in prefixes.items():
        if filename.split('-', 1)[0] == prefix:
            return number
    # Return 6 if no prefix matches (assuming there are only 6 prefixes)
    return 8

In [5]:
# List all files in the directory
files = os.listdir(os.getcwd())

# Create lists to store filenames and corresponding prefix numbers
filenames = []
labels = []

# Iterate through files to collect filename and prefix number
for filename in files:
    filenames.append(filename)
    labels.append(1)

# Create a DataFrame
df = pd.DataFrame({
    'Filename': filenames,
    'Label': labels
})

# # Specify the output Excel file path
# output_excel_file = 'C:\\Users\Alec\Desktop\ESC50\ESC50MFCC'

# # Write DataFrame to Excel
# df.to_excel(output_excel_file, index=False)

# print(f"Data has been written to {output_excel_file}")

In [11]:
def extract_features(chunk, sr):   
    # S = librosa.feature.melspectrogram(y=chunk, sr=sr, n_mels=128)
    # chroma = librosa.feature.chroma_stft(S=np.abs(librosa.stft(S, n_fft=256)), sr=sr)
    mfcc = librosa.feature.mfcc(y=chunk, sr=sr, n_mfcc=13)
    mfcc_delta = librosa.feature.delta(mfcc)
    mfcc_delta2 = librosa.feature.delta(mfcc, order=2)

    # flat_S = S.flatten()
    # flat_chroma = chroma.flatten()
    flat_mfcc = mfcc.flatten()
    flat_mfcc_delta = mfcc_delta.flatten()
    flat_mfcc_delta2 = mfcc_delta2.flatten()
    
    # features = np.hstack((flat_S, flat_chroma, flat_mfcc, flat_mfcc_delta, flat_mfcc_delta2))
    features = np.hstack((flat_mfcc, flat_mfcc_delta, flat_mfcc_delta2))
    
    return features

In [8]:
def norm_get_feats(files):
    feats = []
    for f in files:
        y, sr = librosa.load(f, sr=44100)
        y = librosa.util.normalize(y)
        feats.append(extract_features(y,sr))
    return feats

In [12]:
features = norm_get_feats(files)

In [13]:
df1 = pd.DataFrame(features)

In [14]:
df1

,0,1,2,3,4,5,6,7,8,9,...,3383,3384,3385,3386,3387,3388,3389,3390,3391,3392
0,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-610.498962,-610.498962,-610.498962,-610.498962,-610.498962,-610.498962,-610.498962,-610.498962,-610.498962,-610.498962,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-546.674133,-536.445557,-540.736267,-516.030518,-381.171326,-273.443451,-222.547180,-207.587646,-180.877594,-164.290451,...,0.053612,-0.724129,-0.895783,-0.530420,-0.099159,0.643121,0.643121,0.643121,0.643121,0.643121
9996,-478.731567,-448.632568,-458.094360,-471.814667,-474.857513,-473.972504,-470.670837,-476.166931,-474.885468,-475.191193,...,2.736860,2.360465,1.315711,0.019903,-0.377203,0.046369,0.046369,0.046369,0.046369,0.046369
9997,-484.347076,-467.140442,-482.094971,-492.947754,-496.649658,-500.255157,-502.028900,-507.455200,-513.962891,-514.399536,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9998,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [40]:
# excel_file = 'C:\\Users\Alec\Desktop\SIT_acoustic\labeledAnechoicDataset.xlsx'

# # Read the Excel file into a DataFrame
# df = pd.read_excel(excel_file)

# # Serialize large arrays to JSON strings
# serialized_arrays = [json.dumps(arr.tolist()) for arr in features]

# # Add a new column 'New Column' to the DataFrame and populate it with serialized_arrays
# df['Features'] = serialized_arrays

# # Specify the output Excel file path
# output_excel_file = 'C:\\Users\Alec\Desktop\SIT_acoustic\labeledAnechoicDataset.xlsx'

# # Write the updated DataFrame to Excel
# df.to_excel(output_excel_file, index=False)

# print(f"Data has been written to {output_excel_file}")

Data has been written to C:\Users\Alec\Desktop\SIT_acoustic\labeledAnechoicDataset.xlsx


In [15]:
res = pd.concat([df, df1], axis=1)

In [16]:
res.head()

,Filename,Label,0,1,2,3,4,5,6,7,...,3383,3384,3385,3386,3387,3388,3389,3390,3391,3392
0,1-100032-A-0_chunk1.wav,1,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1-100032-A-0_chunk2.wav,1,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1-100032-A-0_chunk3.wav,1,-610.498962,-610.498962,-610.498962,-610.498962,-610.498962,-610.498962,-610.498962,-610.498962,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1-100032-A-0_chunk4.wav,1,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1-100032-A-0_chunk5.wav,1,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,-1131.370850,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
output_excel_file = 'C:\\Users\Alec\Desktop\ESC50\ESC50_mfcc_d1_d2.csv'

# Write DataFrame to Excel
res.to_csv(output_excel_file, index=False)

In [20]:
ESC50 = pd.read_csv('C:\\Users\Alec\Desktop\ESC50\ESC50_mfcc_d1_d2.csv')

In [21]:
ESC50.head()

,Filename,Label,0,1,2,3,4,5,6,7,...,3383,3384,3385,3386,3387,3388,3389,3390,3391,3392
0,1-100032-A-0_chunk1.wav,1,-1131.37080,-1131.37080,-1131.37080,-1131.37080,-1131.37080,-1131.37080,-1131.37080,-1131.37080,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1-100032-A-0_chunk2.wav,1,-1131.37080,-1131.37080,-1131.37080,-1131.37080,-1131.37080,-1131.37080,-1131.37080,-1131.37080,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1-100032-A-0_chunk3.wav,1,-610.49896,-610.49896,-610.49896,-610.49896,-610.49896,-610.49896,-610.49896,-610.49896,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1-100032-A-0_chunk4.wav,1,-1131.37080,-1131.37080,-1131.37080,-1131.37080,-1131.37080,-1131.37080,-1131.37080,-1131.37080,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1-100032-A-0_chunk5.wav,1,-1131.37080,-1131.37080,-1131.37080,-1131.37080,-1131.37080,-1131.37080,-1131.37080,-1131.37080,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
AnechoicOrig = pd.read_csv("C:\\Users\Alec\Desktop\AnechoicDatasetWork\justMFCCS_deltas.csv")

In [24]:
AnechoicOrig['Label'] = 0

In [28]:
AnechoicOrig.to_csv("C:\\Users\Alec\Desktop\AnechoicDatasetWork\AnechoicMFCCS_deltas_allZero.csv", index=False)

In [29]:
combinedDf =  pd.concat([AnechoicOrig, ESC50], axis=0)

In [33]:
combinedDf.to_csv("C:\\Users\Alec\Desktop\AnechoicDatasetWork\Anechoic_ESC50_mfccsdeltas.csv", index=False)

In [6]:
labels = test['Label']

In [8]:
labels = np.array(labels)

In [10]:
counts = np.ones(8)

In [11]:
for i in labels:
    counts[i] += 1

In [12]:
counts

array([ 933., 1713.,  405., 1241., 1257., 1233., 1237., 1245.])